# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [40]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [41]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cabo San Lucas,MX,22.8909,-109.9124,77.50,44,20,0.00,few clouds
1,1,Lorengau,PG,-2.0226,147.2712,77.47,87,100,8.10,overcast clouds
2,2,Faanui,PF,-16.4833,-151.7500,81.00,73,74,15.05,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,76.75,73,57,2.95,light rain
4,4,Tiksi,RU,71.6872,128.8694,-12.28,98,100,9.26,overcast clouds
5,5,Shingu,JP,33.7333,135.9833,39.79,76,6,6.78,clear sky
6,6,Saskylakh,RU,71.9167,114.0833,-7.87,90,100,17.25,overcast clouds
7,7,Klaksvik,FO,62.2266,-6.5890,34.72,58,54,19.35,broken clouds
8,8,Victoria,HK,22.2855,114.1577,60.33,70,24,8.99,few clouds
9,9,Thompson,CA,55.7435,-97.8558,3.36,78,100,8.05,light snow


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [43]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [44]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

# Display sample data
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cabo San Lucas,MX,22.8909,-109.9124,77.50,44,20,0.00,few clouds
1,1,Lorengau,PG,-2.0226,147.2712,77.47,87,100,8.10,overcast clouds
2,2,Faanui,PF,-16.4833,-151.7500,81.00,73,74,15.05,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,76.75,73,57,2.95,light rain
20,20,Mendoza,AR,-32.8908,-68.8272,87.33,22,0,8.05,clear sky
24,24,Rawson,AR,-43.3002,-65.1023,71.37,43,53,13.69,broken clouds
25,25,Alofi,NU,-19.0595,-169.9187,78.69,89,97,4.61,light rain
26,26,Atuona,PF,-9.8000,-139.0333,77.65,75,7,13.80,clear sky
29,29,De Aar,ZA,-30.6497,24.0123,71.40,35,0,22.39,clear sky
31,31,Butaritari,KI,3.0707,172.7902,80.56,75,46,12.12,scattered clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [45]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cabo San Lucas,MX,22.8909,-109.9124,77.50,44,20,0.00,few clouds
1,1,Lorengau,PG,-2.0226,147.2712,77.47,87,100,8.10,overcast clouds
2,2,Faanui,PF,-16.4833,-151.7500,81.00,73,74,15.05,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,76.75,73,57,2.95,light rain
20,20,Mendoza,AR,-32.8908,-68.8272,87.33,22,0,8.05,clear sky
24,24,Rawson,AR,-43.3002,-65.1023,71.37,43,53,13.69,broken clouds
25,25,Alofi,NU,-19.0595,-169.9187,78.69,89,97,4.61,light rain
26,26,Atuona,PF,-9.8000,-139.0333,77.65,75,7,13.80,clear sky
29,29,De Aar,ZA,-30.6497,24.0123,71.40,35,0,22.39,clear sky
31,31,Butaritari,KI,3.0707,172.7902,80.56,75,46,12.12,scattered clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [46]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [47]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cabo San Lucas,MX,77.50,few clouds,22.8909,-109.9124,
1,Lorengau,PG,77.47,overcast clouds,-2.0226,147.2712,
2,Faanui,PF,81.00,broken clouds,-16.4833,-151.7500,
3,Rikitea,PF,76.75,light rain,-23.1203,-134.9692,
20,Mendoza,AR,87.33,clear sky,-32.8908,-68.8272,
24,Rawson,AR,71.37,broken clouds,-43.3002,-65.1023,
25,Alofi,NU,78.69,light rain,-19.0595,-169.9187,
26,Atuona,PF,77.65,clear sky,-9.8000,-139.0333,
29,De Aar,ZA,71.40,clear sky,-30.6497,24.0123,
31,Butaritari,KI,80.56,scattered clouds,3.0707,172.7902,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [54]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [72]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["features"]["geometry"]["properties"]["details"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Cabo San Lucas - nearest hotel: No hotel found
Lorengau - nearest hotel: No hotel found
Faanui - nearest hotel: No hotel found
Rikitea - nearest hotel: No hotel found
Mendoza - nearest hotel: No hotel found
Rawson - nearest hotel: No hotel found
Alofi - nearest hotel: No hotel found
Atuona - nearest hotel: No hotel found
De Aar - nearest hotel: No hotel found
Butaritari - nearest hotel: No hotel found
Kudahuvadhoo - nearest hotel: No hotel found
Castro - nearest hotel: No hotel found
Basco - nearest hotel: No hotel found
Banda Aceh - nearest hotel: No hotel found
Le Port - nearest hotel: No hotel found
Coahuayana - nearest hotel: No hotel found
Pisco - nearest hotel: No hotel found
Natitingou - nearest hotel: No hotel found
Hilo - nearest hotel: No hotel found
Salalah - nearest hotel: No hotel found
Tres Arroyos - nearest hotel: No hotel found
Chuy - nearest hotel: No hotel found
Alto Paraguai - nearest hotel: No hotel found
Vao - nearest hotel: No hotel found
Mar

Tsabong - nearest hotel: No hotel found
Lufilufi - nearest hotel: No hotel found
Camopi - nearest hotel: No hotel found
Biak - nearest hotel: No hotel found
Marrakesh - nearest hotel: No hotel found
Alyangula - nearest hotel: No hotel found
Capinzal - nearest hotel: No hotel found
Larap - nearest hotel: No hotel found
Kidal - nearest hotel: No hotel found
Pilar - nearest hotel: No hotel found
Muncar - nearest hotel: No hotel found
Rio Grande - nearest hotel: No hotel found
Ahuimanu - nearest hotel: No hotel found
Puerto Baquerizo Moreno - nearest hotel: No hotel found
Jibuti - nearest hotel: No hotel found
Pitimbu - nearest hotel: No hotel found
Poum - nearest hotel: No hotel found
Acari - nearest hotel: No hotel found
Buala - nearest hotel: No hotel found
Cairns - nearest hotel: No hotel found
Dingle - nearest hotel: No hotel found
Boa Vista - nearest hotel: No hotel found
Mareeba - nearest hotel: No hotel found
Diamantina - nearest hotel: No hotel found
Adrar - nearest hotel: No hote

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Cabo San Lucas,MX,77.50,few clouds,22.8909,-109.9124,No hotel found
1,Lorengau,PG,77.47,overcast clouds,-2.0226,147.2712,No hotel found
2,Faanui,PF,81.00,broken clouds,-16.4833,-151.7500,No hotel found
3,Rikitea,PF,76.75,light rain,-23.1203,-134.9692,No hotel found
20,Mendoza,AR,87.33,clear sky,-32.8908,-68.8272,No hotel found
...,...,...,...,...,...,...,...
697,Jalpan,MX,81.97,clear sky,21.2333,-99.4833,No hotel found
698,Businga,CD,76.15,overcast clouds,3.3333,20.8833,No hotel found
699,Samarai,PG,79.93,overcast clouds,-10.6167,150.6667,No hotel found
702,Merauke,ID,76.62,overcast clouds,-8.4667,140.3333,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [73]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="No hotel found")]

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [11]:
# Create the output File (CSV)
# YOUR CODE HERE

# Export the City_Data into a csv
# YOUR CODE HERE

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [12]:
# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Current Description)